# Policy exploration #

This notebook contains the exploration of the possible set of policy alternatives. Given the multi-actor decsion-making arena for water management, it is important to consider what possible preferred policy options could entail for other actors. Hence, exploring the policy options space could profide insights into multiple viable - but general - policy options. They are general in the sense that not one specific objective is centered in the search for a policy, and hence it is as objective as possible to find an effective policy. The latter assumes that all actors agree that minimizing deaths and damage costs is important. 

The found policy options originating in the policy space are compared to the base case results. In so doing, we assess whether any policy will improve the current situation and if a 'group' of policies may be found which have a common characteristic that seems to result in particulary preferred outcomes. For instance, policies that use RfR and dike heightening combined may be very effective in reducing the deaths.

Throughout the notebook, guiding insights and next steps in the analysis are discussed.

In [2]:
# import the libaries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import numpy as np
import pickle

# Import the workbench and model
from ema_workbench import (Model, CategoricalParameter, ScalarOutcome, IntegerParameter, RealParameter)
from ema_workbench import (MultiprocessingEvaluator, Policy, Scenario)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, utilities
from dike_model_function import DikeNetwork 
from ema_workbench import load_results
from problem_formulation import get_model_for_problem_formulation

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

Given the water management simulation model which includes different policy levers (RfR, dike heightening, and evacuation), we use the ema_workbench to randomly generate 100 policies and evaluate their performance under 400 scenarions. This thus results in 40.000 experiments. 

In [2]:
# run the model with randomly generating policy options
dike_model, planning_steps = get_model_for_problem_formulation(5)

def sum_over(*args):
    return sum(args)

# This part is commented out so we only had to run it once
# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results = evaluator.perform_experiments(scenarios=400, policies=100)

# utilities.save_results(results, '../results/400times100PoliciesPF5.csv')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 400 scenarios * 100 policies * 1 model(s) = 40000 experiments
[MainProcess/INFO] 4000 cases completed
[MainProcess/INFO] 8000 cases completed
[MainProcess/INFO] 12000 cases completed
[MainProcess/INFO] 16000 cases completed
[MainProcess/INFO] 20000 cases completed
[MainProcess/INFO] 24000 cases completed
[MainProcess/INFO] 28000 cases completed
[MainProcess/INFO] 32000 cases completed
[MainProcess/INFO] 36000 cases completed
[MainProcess/INFO] 40000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] results saved successfully to C:\Users\hidde\OneDrive\Documenten\EPA\Model Based Decision Making\EPA1361\final assignment\results\400times100PoliciesPF5.csv


In [3]:
from ema_workbench import load_results
experiments, outcomes = load_results('../results/40000scenarios.tar.gz')
outcomes_df = pd.DataFrame.from_dict(outcomes)
outcomes_df.head()

[MainProcess/INFO] results loaded succesfully from C:\Users\irene\OneDrive\Documenten\GitHub\EPA1361\final assignment\results\40000scenarios.tar.gz


,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Dike Investment Costs 2,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2
0,0.0,8.526436e+07,0.0,5.302356e+07,0,0.005742,0.0,3.139880e+07,0.0,1.069181e+07,...,2.568188e+07,0.0,0.0,8.673386e+06,0.0,0.000000e+00,3.948301e+07,0.000000,217800000.0,0.000000
1,0.0,8.526436e+07,0.0,1.357645e+06,0,0.000244,0.0,3.139880e+07,0.0,8.357101e+05,...,2.568188e+07,0.0,0.0,8.673386e+06,0.0,8.432576e+05,3.948301e+07,0.000116,217800000.0,73.837395
2,0.0,8.526436e+07,0.0,2.171279e+07,0,0.002411,0.0,3.139880e+07,0.0,0.000000e+00,...,2.568188e+07,0.0,0.0,8.673386e+06,0.0,0.000000e+00,3.948301e+07,0.000000,217800000.0,0.000000
3,0.0,8.526436e+07,0.0,2.157793e+07,0,0.003005,0.0,3.139880e+07,0.0,0.000000e+00,...,2.568188e+07,0.0,0.0,8.673386e+06,0.0,0.000000e+00,3.948301e+07,0.000000,217800000.0,0.000000
4,0.0,8.526436e+07,0.0,6.413019e+07,0,0.008581,0.0,3.139880e+07,0.0,0.000000e+00,...,2.568188e+07,0.0,0.0,8.673386e+06,0.0,2.566302e+07,3.948301e+07,0.002150,217800000.0,1395.133165


In [11]:
max_range = outcomes_df.max()
min_range = outcomes_df.min()

with open('../results/min_max_range.pickle', 'wb') as f:
    pickle.dump([max_range, min_range], f)